In [15]:
import pandas as pd

org_df = pd.read_csv('parse_tree.csv')
org_df = org_df.set_index(org_df['token_idx'])

org_df

,head_token_idx,label,pos,text,token_idx
token_idx,,,,,
0,4,ADVMOD,ADV,빨리,0
1,4,DOBJ,NOUN,주문,1
2,4,AUX,NOUN,취소,2
3,2,SUFF,AFFIX,해,3
4,4,ROOT,VERB,줘요,4


In [66]:
l = [[4, 3, 0, 1], [4, 3, 2]]

df = pd.DataFrame(l).fillna(-1)
df = df.astype(int)
display(df)

for ll in l:
    display(org_df.loc[ll])

,0,1,2,3
0,4,3,0,1
1,4,3,2,-1


,head_token_idx,label,pos,text,token_idx
token_idx,,,,,
4,4,ROOT,NUM,하나요,4
3,4,ADVCL,VERB,해야,3
0,3,ADVCL,NOUN,환불,0
1,0,SUFF,AFFIX,하려면,1


,head_token_idx,label,pos,text,token_idx
token_idx,,,,,
4,4,ROOT,NUM,하나요,4
3,4,ADVCL,VERB,해야,3
2,3,ADVMOD,ADJ,어떻게,2


In [67]:
import numpy as np

def get_diff_position(df, idx1, idx2):
    diff_p = 0
    _, cols = df.shape
    
    for c_idx in range(cols-1, -1, -1): 
        if df[c_idx][idx1:idx1+1].values != df[c_idx][idx2:idx2+1].values:
            diff_p = c_idx
            
    return diff_p

def get_dependency(df, depth):
    backwards = False
    idxs = []
    for index, row in df.iterrows():
        if backwards:
            idxs.insert(position,row['token_idx'])
        else:
            idxs.append(row['token_idx'])
            
        backwards, position = (True, len(idxs) - 1) if row['label'] in ['SUFF','PTR'] else (False, None)
        
        if row['label'] in ['DOBJ','NSUBJ']:
            es_dict['what']['depth'] = depth
            es_dict['what']['label'] = row['label']
            es_dict['what']['tokens'] = idxs[:]
        elif row['label'] in ['ADVCL']:
            es_dict['why']['depth'] = depth
            es_dict['why']['label'] = row['label']
            es_dict['why']['tokens'] = idxs[:]
        elif row['label'] == 'ROOT':
            es_dict['how']['depth'] = depth
            es_dict['how']['label'] = row['label']
            es_dict['how']['tokens'] = idxs[:]

    return idxs

    
rows, cols = df.shape
prev_dp = cols

es_dict = {
    "when":{
        'depth':None,
        'label':None,
        'tokens':[]
    },
    "what":{
        'depth':None,
        'label':None,
        'tokens':[]
    },
    "how":{
        'depth':None,
        'label':None,
        'tokens':[]
    },
    "why":{
        'depth':None,
        'label':None,
        'tokens':[]
    }
}

for r_idx in range(rows):
    dp = get_diff_position(df, r_idx, r_idx+1)
    cul_cols = [i for i in range(dp,cols)]
    print("diff:",dp)
    
    t = df[cul_cols][r_idx:r_idx+1].values[0][::-1]
    t = t[np.where(t>-1)]
    print(t)
    print(org_df.loc[t])
    get_dependency(org_df.loc[t], dp)
    
    prev_dp = dp
    
display(es_dict)

diff: 2
[1 0]
           head_token_idx  label    pos text  token_idx
token_idx                                              
1                       0   SUFF  AFFIX  하려면          1
0                       3  ADVCL   NOUN   환불          0
diff: 0
[2 3 4]
           head_token_idx   label   pos text  token_idx
token_idx                                              
2                       3  ADVMOD   ADJ  어떻게          2
3                       4   ADVCL  VERB   해야          3
4                       4    ROOT   NUM  하나요          4


{'how': {'depth': 0, 'label': 'ROOT', 'tokens': [2, 3, 4]},
 'what': {'depth': None, 'label': None, 'tokens': []},
 'when': {'depth': None, 'label': None, 'tokens': []},
 'why': {'depth': 0, 'label': 'ADVCL', 'tokens': [2, 3]}}

In [16]:
es_df = pd.read_csv('es_df.csv')
display(es_df)
display(es_df[es_df['kind'] == 'how']['entities'])

,kind,entities,action,branch,morphs,origin
0,what,['주문/Noun'],[],[1],['주문/Noun'],주문
1,how,['취소/Noun'],['하다/Verb'],"[0, 2, 3, 4]","['빨리/Adverb', '취소/Noun', '하다/Verb', '주다/Verb']",빨리 취소해 줘요


1    ['취소/Noun']
Name: entities, dtype: object